In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np

# 1. Load dataset
data = pd.read_csv("house_price.csv") 

# 2. Select features and target
X = data[["size", "bedroom"]]
y = data["price"]

# 3. Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("===== Linear Regression =====")
# 4. LinearRegression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Coefficients
print(f"Coefficients: {lr_model.coef_}")
print(f"Intercept: {lr_model.intercept_}")

# Predictions
y_pred_lr = lr_model.predict(X_test)

# MAE, MSE, RMSE, MAPE
mae_lr = mean_absolute_error(y_test, y_pred_lr)
mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
mape_lr = mean_absolute_percentage_error(y_test, y_pred_lr)

print(f"MAE: {mae_lr:.2f}")
print(f"MSE: {mse_lr:.2f}")
print(f"RMSE: {rmse_lr:.2f}")
print(f"MAPE: {mape_lr:.4f}")

print("\n===== SGD Regressor =====")
# 5. SGDRegressor (requires standardization)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

sgd_model = SGDRegressor(max_iter=1000, tol=1e-3, random_state=42)
sgd_model.fit(X_train_scaled, y_train)

# Coefficients
print(f"Coefficients: {sgd_model.coef_}")
print(f"Intercept: {sgd_model.intercept_}")

# Predictions
y_pred_sgd = sgd_model.predict(X_test_scaled)

# MAE, MSE, RMSE, MAPE
mae_sgd = mean_absolute_error(y_test, y_pred_sgd)
mse_sgd = mean_squared_error(y_test, y_pred_sgd)
rmse_sgd = np.sqrt(mse_sgd)
mape_sgd = mean_absolute_percentage_error(y_test, y_pred_sgd)

print(f"MAE: {mae_sgd:.2f}")
print(f"MSE: {mse_sgd:.2f}")
print(f"RMSE: {rmse_sgd:.2f}")
print(f"MAPE: {mape_sgd:.4f}")


===== Linear Regression =====
Coefficients: [   143.21853204 -13512.56442597]
Intercept: 84763.62252219388
MAE: 72334.75
MSE: 8610424544.78
RMSE: 92792.37
MAPE: 0.1746

===== SGD Regressor =====
Coefficients: [106535.91023723 -10274.95128909]
Intercept: [323155.83200156]
MAE: 72124.61
MSE: 8595003325.39
RMSE: 92709.24
MAPE: 0.1740


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from joblib import dump, load

# Load the image and extract features
def extract_hog(img_path, size=(128, 128)):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, size)
    hog = cv2.HOGDescriptor()
    return hog.compute(img).flatten()

# Prepare the dataset
def load_dataset(folder):
    X, y = [], []
    for label, class_name in enumerate(['Cat', 'Dog']):
        path = os.path.join(folder, class_name)
        for file in os.listdir(path):
            img_path = os.path.join(path, file)
            try:
                features = extract_hog(img_path)
                X.append(features)
                y.append(label)
            except:
                continue
    return np.array(X), np.array(y)

# Dataset path
train_dir = r"D:\desktop\CVI\codes\train"
X, y = load_dataset(train_dir)

# Split the training set and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the SVM model
model = SVC(kernel='linear', probability=True)
model.fit(X_train, y_train)

# Test accuracy
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['Cat', 'Dog']))

#save
dump(model, 'cat_dog_svm_model.joblib')


              precision    recall  f1-score   support

         Cat       0.68      0.67      0.68       199
         Dog       0.68      0.69      0.68       201

    accuracy                           0.68       400
   macro avg       0.68      0.68      0.68       400
weighted avg       0.68      0.68      0.68       400



['cat_dog_svm_model.joblib']

In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# load data
train_df = pd.read_csv("mnist_train.csv")
test_df = pd.read_csv("mnist_test.csv")

# Sampling and splitting.Normalization
train_df_sampled = train_df.sample(n=10000, random_state=42)

X_train = train_df_sampled.iloc[:, 1:].to_numpy(dtype=np.float32) / 255.0
y_train = train_df_sampled.iloc[:, 0].to_numpy()

X_test = test_df.iloc[:, 1:].to_numpy(dtype=np.float32) / 255.0
y_test = test_df.iloc[:, 0].to_numpy()

# Build and train the model
mlp = MLPClassifier(
    hidden_layer_sizes=(128, 64),
    max_iter=100,
    early_stopping=True,
    random_state=42
)
mlp.fit(X_train, y_train)

# Predict and evaluate
y_pred = mlp.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 95.34%

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       980
           1       0.98      0.99      0.98      1135
           2       0.94      0.95      0.95      1032
           3       0.94      0.95      0.94      1010
           4       0.95      0.94      0.95       982
           5       0.97      0.93      0.95       892
           6       0.94      0.96      0.95       958
           7       0.96      0.95      0.96      1027
           8       0.94      0.94      0.94       974
           9       0.95      0.94      0.94      1009

    accuracy                           0.95      9999
   macro avg       0.95      0.95      0.95      9999
weighted avg       0.95      0.95      0.95      9999

